In [143]:
import pandas
import requests
import io
import zipfile
import json

In [79]:
def get_zipped_csv_from_url(url : str, csv_name : str, num_rows : int) -> pandas.DataFrame:
    web_file = requests.get(url=url)
    zip = zipfile.ZipFile(io.BytesIO(web_file.content))
    #data = pandas.read_csv(zip.open(csv_name), nrows=num_rows,usecols=['Date received','Issue'])
    data = pandas.read_csv(zip.open(csv_name), nrows=num_rows)
    return pandas.DataFrame(data)

    #print(df)
df = get_zipped_csv_from_url("https://files.consumerfinance.gov/ccdb/complaints.csv.zip","complaints.csv",10)

In [80]:
id = "1228393702244134912"
endpoint = "https://api.twitter.com/2/tweets/"+id

def get_credentials(path: str) -> str:
    file = open(credential_path, "r")
    token = file.readline()
    bearer_token = "Bearer " + token
    return bearer_token


    
credential_path = "/Users/at/API Credentials/TwitterCreds.txt"
file = open(credential_path, "r")
#read first line for bearer
token = file.readline()
bearer_token = "Bearer " + token
headers = {"Authorization" : bearer_token}
response = requests.get(endpoint,headers=headers).json()
print(response)


{'data': {'id': '1228393702244134912', 'text': 'What did the developer write in their Valentine’s card?\n  \nwhile(true) {\n    I = Love(You);  \n}'}}


In [319]:
####-----DOES NOT WORK YET--- NEED upgraded access levels for Twitter API
search_url = "https://api.twitter.com/2/tweets/search/all"
query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}
credential_path = "/Users/at/API Credentials/TwitterCreds.txt"
creds = get_credentials(credential_path)
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {creds}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r


def connect_to_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()
response = connect_to_endpoint(search_url, query_params)
#print(response.json())

def twitter_request(str: credential_path, url: str, query_params :dict):
    creds = get_credentials(credential_path)




    

401


Exception: (401, '{\n  "title": "Unauthorized",\n  "type": "about:blank",\n  "status": 401,\n  "detail": "Unauthorized"\n}')

In [313]:
#--Twitter Auth and Request
# Twitter request Factory creates a factory that creates twitter api requests
# The factory is in charge of handling the credentials by retrieving the credentials (bearer token)
# from file and determining the url of the endpoint for the request.
# The actual request just requires a query
# Create a new factory for each request type
# I guess this is way to not have to retrieve credentials everytime you need to send a request

class twitter_request_factory:
       
    bearer_token : str
    url : str
    request_type : str
    def __init__(self, url: str, credential_path, request_type):
        self.url = url
        self.request_type = request_type
        self.bearer_token = get_credentials(credential_path)

    def get_credentials(path: str) -> str:
        file = open(credential_path, "r")
        token = file.readline()
        bearer_token = "Bearer " + token
        return bearer_token
    
    def bearer_oauth(self,r):
        """
        Method required by bearer token authentication.
        """
        token = bearer_oauth
        r.headers["Authorization"] = f"Bearer {token}"
        r.headers["User-Agent"] = self.request_type
        return r
            
    def twitter_request_all(self, query_params):
        response = requests.request("GET", self.url, auth=self.bearer_oauth, params=query_params)
        #print(response.status_code)
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)
        return response.json()
   
    def twitter_request_get_recent_tweets(self, query_params):
        headers = {"Authorization" : self.bearer_token}
        response = requests.request("GET", self.url, params = query_params, headers=headers)
        #print(response.status_code)
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)
        return response
    
twit_factory = twitter_request_factory("https://api.twitter.com/2/tweets/search/all", "/Users/at/API Credentials/TwitterCreds.txt","v2FullArchiveSearchPython")

#print(twit_factory.twitter_request_all({'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}))

In [258]:
#----This block creates a recent_tweet_factory that is used to create recent tweet request based
# on one set of credentials that are stored in the factory
# Each time the loop iterates it produces a new request with the "next_token" from the previous request along with the expected payload

def tweet_reader():
    recent_tweet_factory = twitter_request_factory("https://api.twitter.com/2/tweets/search/recent", "/Users/at/API Credentials/TwitterCreds.txt","NONE")
    query_params = {'query':'cruise'}
    recent_tweet_request = recent_tweet_factory.twitter_request_get_recent_tweets(query_params)
    response = json.dumps(recent_tweet_request.json(),indent=4, sort_keys=True)
    print(response)
    response_dict = json.loads(response)
    next_request_token = response_dict["meta"]
    next_request_token = next_request_token['next_token']
    i = 0
    while i < 10:
        new_query_params = {'query':'cruise', 'next_token':next_request_token}
        recent_tweet_request = recent_tweet_factory.twitter_request_get_recent_tweets(query_params)
        response = json.dumps(recent_tweet_request.json(),indent=4, sort_keys=True)
        print(response)
        next_request_token = response_dict["meta"]
        next_request_token = next_request_token['next_token']
        tweet_data = response_dict["data"]
        
        i+=1


tweet_reader()

200
{
    "data": [
        {
            "id": "1481420388869361665",
            "text": "@platinum_cruise \u30af\u30de\u3055\u3093\u3001\u304a\u306f\u3088\u3046\u3054\u3056\u3044\u307e\u3059\ud83c\udf40\n\n\u3053\u306e\u6642\u671f\u3089\u3057\u3044\u771f\u51ac\u306e\u5bd2\u3055\u304c\u7d9a\u304d\u307e\u3059:;(\u2229\u00b4\ufe4f`\u2229);:\u6696\u304b\u304f\u3057\u3066\u904e\u3054\u3057\u4e0b\u3055\u3044\u306d\ud83c\udf40 \ud83c\udf40   \n\n\u4eca\u65e5\u3082\u3088\u308d\u3057\u304f\u304a\u9858\u3044\u3057\u307e\u3059\ud83d\ude47\u200d\u2642\ufe0f\ud83c\udf40\n\n\u826f\u3044\u65e5\u3067\u3042\u308a\u307e\u3059\u3088\u3046\u306b\u2615\ufe0f\u266a\ud83c\udf40\n\n\u4e00\u65e5\u4e00\u7948\ud83c\uddef\ud83c\uddf5\n#\u62c9\u81f4\u88ab\u5bb3\u8005\u5168\u54e1\u596a\u9084 \n#\u7279\u5b9a\u5931\u8e2a\u8005\u5168\u54e1\u596a\u9084\n\n\uff76\uff97\uff8c\uff84\uff8b\uff96\uff78\uff7f\uff73 (\u6a3a\u592a\u6bd4\u7ffc\u8349)\uff75\uff75\uff8a\uff9e\uff7a\u79d1 https://t.co/gQW6DCTIEB"
        },
   

In [326]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017")
database = client["MainDatabase"]
cruise_tweets = database["CruiseTweets"]

#----This block creates a recent_tweet_factory that is used to create recent tweet request based
# on one set of credentials that are stored in the factory
# Each time the loop iterates it produces a new request with the "next_token" from the previous request along with the expected payload

def tweet_reader(iterations: int, query: str):
    recent_tweet_factory = twitter_request_factory("https://api.twitter.com/2/tweets/search/recent", "/Users/at/API Credentials/TwitterCreds.txt","NONE")
    query_params = {'query': query}
    recent_tweet_request = recent_tweet_factory.twitter_request_get_recent_tweets(query_params)
    response = json.dumps(recent_tweet_request.json(),indent=4, sort_keys=True)
    #print(response)
    response_dict = json.loads(response)
    next_request_token = response_dict["meta"]
    next_request_token = next_request_token['next_token']
    i = 0
    while i <= iterations:
        new_query_params = {'query':'cruise', 'next_token':next_request_token}
        recent_tweet_request = recent_tweet_factory.twitter_request_get_recent_tweets(new_query_params)
        response = json.dumps(recent_tweet_request.json(),indent=4, sort_keys=True)      
        #print(response)
        response_dict = json.loads(response)
        next_request_token = response_dict["meta"]
        next_request_token = next_request_token['next_token']
        tweet_data = response_dict["data"]
        cruise_tweets.insert_many(tweet_data)
        #for entry in tweet_data: 
            #cruise_tweets.insert_one(entry)
        i+=1






In [325]:
tweet_reader(1,'covid,cruise')
for x in cruise_tweets.find():
    print(x)

{'_id': ObjectId('61e0e062afcd9cd53a1493cd'), 'id': '1481815854644080640', 'text': "India successfully test-fires naval variant of BrahMos cruise missile; Rajnath lauds navy's 'mission readiness'\nhttps://t.co/I9RU7P4NQt\n\nvia NaMo App https://t.co/e2UIiS3T26"}
{'_id': ObjectId('61e0e062afcd9cd53a1493ce'), 'id': '1481815818497400832', 'text': 'The latest Cruise Planners Travel News! #CruisePlanners #travel #cruise #honeymoon #vacation https://t.co/8KX2e88NVH Thanks to @MirrorCeleb @OK_Magazine #travel #news'}
{'_id': ObjectId('61e0e062afcd9cd53a1493cf'), 'id': '1481815806086299649', 'text': 'RT @bachinunez_nuez: Lo último,  el Tom Cruise del Este \u2066@MiguelPrietoCDE\u2069 Alias #LadronzueloDePandemias manda construir Columbarios por un…'}
{'_id': ObjectId('61e0e062afcd9cd53a1493d0'), 'id': '1481815801392996353', 'text': '@foolerrant I cringed reading this because... Yeah. No. I never wanted to do a cruise before the plague hit. Now? Now you\'d have to pay me an awful lot of money. 

In [324]:
cruise_tweets.delete_many({})